In [1]:
import os
import pandas as pd
import json

In [3]:
input_path = "../../public/data/geo/raw"
output_path = "../../public/data/geo/canon"

POP_THRESHOLDS = [
    1000000, 200000, 100000,
    50000, 10000, 1000, 0
]

COUNT_THRESHOLDS = [
    10, 25, 50, 100, 250, 500, 1000
]

slugs = []

for o in os.listdir(input_path):
    if not o.endswith(".csv"):
        continue

    file_slug = o.replace(".csv", "")
    df = pd.read_csv(os.path.join(input_path, o), encoding="utf-8", delimiter=";", header=None)

    file_slugs = {}
    D = {}

    for r in df.to_dict("records"):
        name = f"{r[0]} [{r[5]}] |{r[2]}|{r[3]} #{r[1]}#"
        D[name] = [r[7], r[6], r[4]]

        for pop in POP_THRESHOLDS:
            if r[4] >= pop:
                slug_key = f"{file_slug}"
                sub_key = f"{file_slug}-{r[1]}"

                if slug_key not in file_slugs:
                    file_slugs[slug_key] = {}
                if pop not in file_slugs[slug_key]:
                    file_slugs[slug_key][pop] = 0
                if sub_key not in file_slugs:
                    file_slugs[sub_key] = {}
                if pop not in file_slugs[sub_key]:
                    file_slugs[sub_key][pop] = 0
                file_slugs[slug_key][pop] += 1

                if file_slug in ["world", "us", "australia", "canada"]:
                    file_slugs[sub_key][pop] += 1

    for file_slug in file_slugs:
        if "XXX" in file_slug or file_slug.count("-") > 1:
            continue
        pops = file_slugs[file_slug]
        curr_count = 0
        for pop in POP_THRESHOLDS[::-1]:
            if (curr_count == 0 and pops.get(pop, 0) >= 20) or (curr_count - pops.get(pop, curr_count) >= 20 and pops.get(pop, 0) >= 20):
                curr_count = pops[pop]
                slugs.append(f"{file_slug}-{pop}")

        count = pops.get(0, 0)
        for count_threshold in COUNT_THRESHOLDS:
            if count >= count_threshold:
                slugs.append(f"{file_slug}-top{count_threshold}")
    
    with open(os.path.join(output_path, o.replace(".csv", ".json")), "w", encoding="utf-8") as f:
        json.dump(D, f, ensure_ascii=False, indent=4)


In [4]:
other_path = "../../public/data/geo/canon"
for o in os.listdir(other_path):
    o_slug = o.replace(".json", "")
    slugs.append(o_slug)

In [5]:
with open(os.path.join("../../src/data", "geo-slugs.json"), "w", encoding="utf-8") as f:
    json.dump(slugs, f, ensure_ascii=False, indent=4)